In [1]:
!pip install fastai -Uqq
!pip install pytorch-lightning -Uqq
!pip install pytorch-forecasting -Uqq

     |████████████████████████████████| 189 kB 4.1 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 527 kB 3.2 MB/s 
     |████████████████████████████████| 396 kB 34.4 MB/s 
     |████████████████████████████████| 596 kB 35.4 MB/s 
     |████████████████████████████████| 133 kB 36.4 MB/s 
     |████████████████████████████████| 829 kB 32.7 MB/s 
     |████████████████████████████████| 952 kB 32.3 MB/s 
     |████████████████████████████████| 1.1 MB 34.2 MB/s 
     |████████████████████████████████| 94 kB 2.8 MB/s 
     |████████████████████████████████| 271 kB 38.3 MB/s 
     |████████████████████████████████| 144 kB 36.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 113

In [1]:
from fastai.tabular.all import *
import seaborn as sns
from tqdm import tqdm

In [ ]:
import os
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

os.chdir("../../..")

# Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = Path('/content/drive/MyDrive/colab_notebooks/algovera_defi')

# 1. Load Data

In [4]:
df = pd.read_csv(path/'compound.csv')
df["Date"] = pd.to_datetime(df["Timestamp"], unit='s', origin='unix')

tokens = df["Token"].unique()
tokens

array(['DAI', 'USDC', 'USDT', 'ETH'], dtype=object)

In [5]:
df = df.drop_duplicates(['Timestamp', 'Token'])
counts = pd.DataFrame(df['Timestamp'].value_counts()).reset_index()
counts.columns = ['Timestamp', 'Counts']
df = df.merge(counts, on='Timestamp')

In [6]:
df = df[df['Counts'] == 4].reset_index(drop=True).drop('Counts', axis=1)

In [7]:
df1 = pd.DataFrame()
for tok in tokens:
    df_tok = df[df['Token']==tok]
    df_tok = df_tok.drop(['Token', 'Date'], axis=1)

    col_names = []
    for col in df_tok.columns:
        if col == 'Timestamp':
            col_names.append(f'{col}')
        else:
            col_names.append(f'{tok}_{col}')
        
    df_tok.columns = col_names
    #df_tok = df_tok.set_index('Timestamp', drop=True)
    
    if df1.empty:
        df1 = df_tok
    else:
        df1 = pd.merge(df1, df_tok, on='Timestamp')

In [8]:
df1.sort_values('Timestamp', inplace=True)
df1["Date"] = pd.to_datetime(df1["Timestamp"], unit='s', origin='unix')
df1['Timediff'] = df1['Timestamp'].shift(-1) - df1['Timestamp'] 

In [9]:
df1

,Timestamp,DAI_Borrowing Rate,DAI_Deposit Rate,DAI_Borrow Volume,DAI_Supply Volume,USDC_Borrowing Rate,USDC_Deposit Rate,USDC_Borrow Volume,USDC_Supply Volume,USDT_Borrowing Rate,USDT_Deposit Rate,USDT_Borrow Volume,USDT_Supply Volume,ETH_Borrowing Rate,ETH_Deposit Rate,ETH_Borrow Volume,ETH_Supply Volume,Date,Timediff
0,1609471800,0.073195,0.050982,1.069964e+09,6.196481e+10,0.087046,0.066993,7.285430e+08,4.063042e+10,0.099588,0.077548,6.430536e+07,3.696225e+09,0.022952,0.000489,30553.654354,5.663257e+07,2021-01-01 03:30:00,1800.0
1,1609473600,0.073101,0.050912,1.069961e+09,6.197050e+10,0.087053,0.066998,7.285469e+08,4.063019e+10,0.094890,0.073569,6.407877e+07,3.700299e+09,0.022952,0.000489,30553.703955,5.663353e+07,2021-01-01 04:00:00,1800.0
2,1609475400,0.073061,0.050882,1.069972e+09,6.197354e+10,0.087058,0.067003,7.285528e+08,4.063019e+10,0.085767,0.065933,6.399418e+07,3.729213e+09,0.022951,0.000489,30553.830472,5.664534e+07,2021-01-01 04:30:00,1800.0
3,1609477200,0.073436,0.051161,1.070496e+09,6.197966e+10,0.086921,0.066889,7.285713e+08,4.063660e+10,0.072946,0.055500,6.317162e+07,3.730028e+09,0.022952,0.000489,30553.786270,5.663860e+07,2021-01-01 05:00:00,1800.0
4,1609479000,0.067829,0.047015,1.070566e+09,6.234580e+10,0.086312,0.066383,7.285755e+08,4.066162e+10,0.057764,0.043239,6.256095e+07,3.752927e+09,0.022952,0.000489,30553.841412,5.663440e+07,2021-01-01 05:30:00,1800.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13761,1639494000,0.046639,0.031456,3.669928e+09,2.109559e+11,0.040312,0.025658,2.588043e+09,1.666908e+11,0.039243,0.024325,6.072526e+08,4.162081e+10,0.027233,0.000765,53162.061654,7.448759e+07,2021-12-14 15:00:00,3600.0
13762,1639497600,0.046638,0.031456,3.669944e+09,2.109575e+11,0.040311,0.025657,2.587365e+09,1.666495e+11,0.039302,0.024397,6.075364e+08,4.157885e+10,0.027234,0.000765,53173.698145,7.448781e+07,2021-12-14 16:00:00,3600.0
13763,1639501200,0.046638,0.031456,3.669965e+09,2.109580e+11,0.040320,0.025668,2.585915e+09,1.665187e+11,0.039309,0.024405,6.076393e+08,4.157875e+10,0.027240,0.000767,53163.371786,7.435258e+07,2021-12-14 17:00:00,3600.0
13764,1639504800,0.046639,0.031457,3.669982e+09,2.109535e+11,0.040532,0.025937,2.586157e+09,1.656795e+11,0.039307,0.024403,6.076091e+08,4.157863e+10,0.027247,0.000768,53163.546809,7.423226e+07,2021-12-14 18:00:00,1800.0


# 2. Train/test split

In [10]:
target_token = 'DAI'
target_column = f'{target_token}_Borrowing Rate'
target_window = 10
def get_target(row):

    try:
        target = df1[df1['Timestamp'] == row['Timestamp'] + 1800.0*target_window][target_column].values[0]
    except:
        target = np.NaN
    
    return target

In [11]:
df1[f'{target_token}_Target'] =  df1.apply(lambda x: get_target(x), axis=1)
df1 = df1.dropna()
#df1 = df1.drop(['Timestamp', 'Date', 'Timediff'], axis=1)
#df1 = df1.drop(['Timediff'], axis=1)

# PyTorch Forecasting Model

In [12]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

In [42]:
from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import RMSE, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [14]:
df1 = df1.reset_index(drop=True)

In [15]:
#the TimeSeriesDataset requires a constant value for group_ids
df1['Dummy'] = 'Defi'

In [16]:
df1['time_idx'] = (df1['Timestamp'] - df1['Timestamp'].min())/1800
df1['time_idx'] = df1['time_idx'].map(int)

In [17]:
df1.columns

Index(['Timestamp', 'DAI_Borrowing Rate', 'DAI_Deposit Rate',
       'DAI_Borrow Volume', 'DAI_Supply Volume', 'USDC_Borrowing Rate',
       'USDC_Deposit Rate', 'USDC_Borrow Volume', 'USDC_Supply Volume',
       'USDT_Borrowing Rate', 'USDT_Deposit Rate', 'USDT_Borrow Volume',
       'USDT_Supply Volume', 'ETH_Borrowing Rate', 'ETH_Deposit Rate',
       'ETH_Borrow Volume', 'ETH_Supply Volume', 'Date', 'Timediff',
       'DAI_Target', 'Dummy', 'time_idx'],
      dtype='object')

In [18]:
df1 = df1.drop(['Timediff', 'Timestamp', 'Date'], axis=1)

In [19]:
feature_cols = ['DAI_Borrowing Rate', 'DAI_Deposit Rate', 'DAI_Borrow Volume',
                'DAI_Supply Volume', 'USDC_Borrowing Rate', 'USDC_Deposit Rate',
                'USDC_Borrow Volume', 'USDC_Supply Volume', 'USDT_Borrowing Rate',
                'USDT_Deposit Rate', 'USDT_Borrow Volume', 'USDT_Supply Volume',
                'ETH_Borrowing Rate', 'ETH_Deposit Rate', 'ETH_Borrow Volume',
                'ETH_Supply Volume']

In [20]:
int(df1['time_idx'].max()*0.8), df1['time_idx'].max()

(13340, 16676)

In [98]:
max_prediction_length = 1
max_encoder_length = 24 #max_encoder_length (int) : maximum length to encode. This is the maximum history length used by the time series dataset.

training_cutoff = int(df1['time_idx'].max()*0.8)

training = TimeSeriesDataSet(
    df1[lambda x: x.time_idx <= training_cutoff],
    group_ids=['Dummy'],
    time_idx="time_idx",
    target=f"{target_token}_Target",
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=6,
    max_prediction_length=12,
    time_varying_known_categoricals=[],
    time_varying_unknown_categoricals=[],
    time_varying_known_reals=feature_cols,
#    time_varying_unknown_reals=[f"{target_token}_Target"],
    time_varying_unknown_reals=[],
    allow_missing_timesteps=True,
    target_normalizer=None
)

In [99]:
training.data['target'][0].shape

torch.Size([10704])

In [97]:
len(df1[df1['time_idx'] <= training_cutoff])

10704

In [104]:
validation = TimeSeriesDataSet(
    df1[lambda x: x.time_idx > training_cutoff],
    group_ids=['Dummy'],
    time_idx="time_idx",
    target=f"{target_token}_Target",
    min_encoder_length=max_encoder_length // 2,  # keep encoder length long (as it is in the validation set)
    max_encoder_length=max_encoder_length,
    min_prediction_length=6,
    max_prediction_length=12,
    time_varying_known_categoricals=[],
    time_varying_unknown_categoricals=[],
    time_varying_known_reals=feature_cols,
#    time_varying_unknown_reals=[f"{target_token}_Target"],
    time_varying_unknown_reals=[],
    allow_missing_timesteps=True,
    target_normalizer=None
)

# create dataloaders for model
batch_size = 32  # set this between 32 to 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=0)

In [106]:
'''
First entry is x, a dictionary of tensors with the entries (and shapes in brackets)
    encoder_cat (batch_size x n_encoder_time_steps x n_features): long tensor of encoded categoricals for encoder
    encoder_cont (batch_size x n_encoder_time_steps x n_features): float tensor of scaled continuous variables for encoder
    encoder_target (batch_size x n_encoder_time_steps or list thereof with each entry for a different target): float tensor with unscaled continous target or encoded categorical target, list of tensors for multiple targets
    encoder_lengths (batch_size): long tensor with lengths of the encoder time series. No entry will be greater than n_encoder_time_steps
    decoder_cat (batch_size x n_decoder_time_steps x n_features): long tensor of encoded categoricals for decoder
    decoder_cont (batch_size x n_decoder_time_steps x n_features): float tensor of scaled continuous variables for decoder
    decoder_target (batch_size x n_decoder_time_steps or list thereof with each entry for a different target): float tensor with unscaled continous target or encoded categorical target for decoder - this corresponds to first entry of y, list of tensors for multiple targets
    decoder_lengths (batch_size): long tensor with lengths of the decoder time series. No entry will be greater than n_decoder_time_steps
    group_ids (batch_size x number_of_ids): encoded group ids that identify a time series in the dataset
    target_scale (batch_size x scale_size or list thereof with each entry for a different target): parameters used to normalize the target. Typically these are mean and standard deviation. Is list of tensors for multiple targets.

Second entry is y, a tuple of the form (target, weight)
    target (batch_size x n_decoder_time_steps or list thereof with each entry for a different target): unscaled (continuous) or encoded (categories) targets, list of tensors for multiple targets
    weight (None or batch_size x n_decoder_time_steps): weight
'''
for x,y in iter(train_dataloader):
    print(x)
    break

{'encoder_cat': tensor([], size=(32, 24, 0), dtype=torch.int64), 'encoder_cont': tensor([[[ 0.8333, -0.8268, -0.8134,  ..., -0.3441,  0.4472,  1.2181],
         [ 0.8333, -0.8276, -0.8145,  ..., -0.3441,  0.4473,  1.2181],
         [ 0.8333, -0.8276, -0.8145,  ..., -0.3441,  0.4473,  1.2181],
         ...,
         [ 0.8333, -0.8277, -0.8147,  ..., -0.3502,  0.4385,  1.2167],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 0.9167, -0.5404, -0.4805,  ..., -0.2345, -0.5769, -0.8020],
         [ 0.9167, -0.5583, -0.4968,  ..., -0.2854, -0.6227, -0.8035],
         [ 0.9167, -0.5762, -0.5129,  ..., -0.2707, -0.6129, -0.8076],
         ...,
         [ 0.9167, -0.5959, -0.5308,  ..., -0.1242, -0.4565, -0.7688],
         [ 0.9167, -0.5961, -0.5309,  ..., -0.0950, -0.4562, -0.8022],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[ 1.0000, -0.0938, -0.0706

In [107]:
#we have no categorical features 
x['encoder_cat']

tensor([], size=(32, 24, 0), dtype=torch.int64)

In [108]:
#cont features
x['encoder_cont'].shape

torch.Size([32, 24, 17])

In [109]:
training.reals, len(training.reals) 

(['encoder_length',
  'DAI_Borrowing Rate',
  'DAI_Deposit Rate',
  'DAI_Borrow Volume',
  'DAI_Supply Volume',
  'USDC_Borrowing Rate',
  'USDC_Deposit Rate',
  'USDC_Borrow Volume',
  'USDC_Supply Volume',
  'USDT_Borrowing Rate',
  'USDT_Deposit Rate',
  'USDT_Borrow Volume',
  'USDT_Supply Volume',
  'ETH_Borrowing Rate',
  'ETH_Deposit Rate',
  'ETH_Borrow Volume',
  'ETH_Supply Volume'],
 17)

In [110]:
x['encoder_target'][0], x['encoder_target'][0].shape 

(tensor([0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437,
         0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437,
         0.0437, 0.0437, 0.0437, 0.0437, 0.0000, 0.0000]), torch.Size([24]))

In [111]:
x['encoder_target'][2], x['encoder_target'][2].shape

(tensor([0.0675, 0.0663, 0.0794, 0.0807, 0.0808, 0.0808, 0.0808, 0.0808, 0.0808,
         0.0690, 0.0690, 0.0695, 0.0700, 0.0699, 0.0698, 0.0698, 0.0698, 0.0698,
         0.0698, 0.0698, 0.0699, 0.0697, 0.0696, 0.0696]), torch.Size([24]))

In [112]:
x['encoder_lengths']

tensor([22, 23, 24, 23, 24, 24, 24, 23, 24, 24, 24, 24, 24, 24, 24, 23, 24, 24,
        24, 24, 24, 24, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24])

In [113]:
x['decoder_cat']

tensor([], size=(32, 12, 0), dtype=torch.int64)

In [114]:
x['decoder_cont'].shape

torch.Size([32, 12, 17])

In [115]:
x['decoder_target']

tensor([[0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437, 0.0437,
         0.0437, 0.0437, 0.0437],
        [0.0561, 0.0561, 0.0561, 0.0547, 0.0581, 0.0583, 0.0599, 0.0603, 0.0603,
         0.0604, 0.0616, 0.0634],
        [0.0697, 0.0698, 0.0699, 0.0699, 0.0697, 0.0697, 0.0696, 0.0695, 0.0695,
         0.0696, 0.0697, 0.0694],
        [0.0791, 0.0802, 0.0802, 0.0802, 0.0803, 0.0804, 0.0804, 0.0804, 0.0803,
         0.0797, 0.0797, 0.0797],
        [0.0422, 0.0422, 0.0422, 0.0422, 0.0422, 0.0422, 0.0422, 0.0422, 0.0422,
         0.0422, 0.0422, 0.0422],
        [0.0439, 0.0439, 0.0439, 0.0439, 0.0439, 0.0439, 0.0439, 0.0439, 0.0438,
         0.0438, 0.0438, 0.0438],
        [0.1233, 0.1233, 0.1235, 0.1235, 0.1235, 0.1235, 0.1235, 0.1238, 0.1238,
         0.1238, 0.1238, 0.1238],
        [0.0387, 0.0387, 0.0388, 0.0388, 0.0388, 0.0388, 0.0388, 0.0388, 0.0388,
         0.0388, 0.0388, 0.0388],
        [0.1629, 0.1631, 0.1643, 0.1634, 0.1618, 0.1598, 0.1559, 0.1804, 0.1837,

In [116]:
x['decoder_lengths']

tensor([12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12])

In [117]:
x['target_scale']

tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]], dtype=torch.float64)

In [118]:
# calculate baseline mean absolute error, i.e. predict next value as the last available value from the history
actuals = torch.cat([y for x, (y, weight) in iter(val_dataloader)])
baseline_predictions = Baseline().predict(val_dataloader)
(actuals - baseline_predictions).abs().mean().item()

nan

In [119]:
# configure network and trainer
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

In [120]:
trainer = pl.Trainer(
    max_epochs=30,
    gpus=1,
    weights_summary="top",
    gradient_clip_val=0.1,
    limit_train_batches=30,  
    callbacks=[lr_logger, early_stop_callback],
    logger=logger,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [121]:
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=16,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=6,
    output_size=7,
    loss=QuantileLoss(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)

In [122]:
tft

TemporalFusionTransformer(
  (loss): QuantileLoss()
  (logging_metrics): ModuleList(
    (0): SMAPE()
    (1): MAE()
    (2): RMSE()
    (3): MAPE()
  )
  (input_embeddings): MultiEmbedding(
    (embeddings): ModuleDict()
  )
  (prescalers): ModuleDict(
    (encoder_length): Linear(in_features=1, out_features=6, bias=True)
    (DAI_Borrowing Rate): Linear(in_features=1, out_features=6, bias=True)
    (DAI_Deposit Rate): Linear(in_features=1, out_features=6, bias=True)
    (DAI_Borrow Volume): Linear(in_features=1, out_features=6, bias=True)
    (DAI_Supply Volume): Linear(in_features=1, out_features=6, bias=True)
    (USDC_Borrowing Rate): Linear(in_features=1, out_features=6, bias=True)
    (USDC_Deposit Rate): Linear(in_features=1, out_features=6, bias=True)
    (USDC_Borrow Volume): Linear(in_features=1, out_features=6, bias=True)
    (USDC_Supply Volume): Linear(in_features=1, out_features=6, bias=True)
    (USDT_Borrowing Rate): Linear(in_features=1, out_features=6, bias=True)
   

In [123]:
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")

Number of parameters in network: 32.7k


In [124]:
trainer.fit(
    tft,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:736: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  "`trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6."
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 204   
4  | static_variable_selection          | VariableSelectionNetwork        | 400   
5  | encoder_variable_select

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (30) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [125]:
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [126]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean()

tensor(0.0243, dtype=torch.float64)